#Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Import Ultralytics

In [ ]:
!nvidia-smi

Tue Apr  9 17:02:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.8/778.8 kB 12.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

# Convert DOTA to YOLO OBB

In [ ]:
!pip install tqdm

In [ ]:
import cv2

In [ ]:
from pathlib import Path
from tqdm import tqdm
import cv2

def my_convert_dota_to_yolo_obb(dota_root_path: str):
    """
    Converts DOTA dataset annotations to YOLO OBB (Oriented Bounding Box) format.

    The function processes images in the 'train' and 'val' folders of the DOTA dataset. For each image, it reads the
    associated label from the original labels directory and writes new labels in YOLO OBB format to a new directory.

    Args:
        dota_root_path (str): The root directory path of the DOTA dataset.

    Example:
        ```python
        from ultralytics.data.converter import convert_dota_to_yolo_obb

        convert_dota_to_yolo_obb('path/to/DOTA')
        ```

    Notes:
        The directory structure assumed for the DOTA dataset:

            - DOTA
                ├─ images
                │   ├─ train
                │   └─ val
                └─ labels
                    ├─ train_original
                    └─ val_original

        After execution, the function will organize the labels into:

            - DOTA
                └─ labels
                    ├─ train
                    └─ val
    """
    dota_root_path = Path(dota_root_path)

    # Class names to indices mapping
    class_mapping = {
        "pedestrian": 0,
        "people": 1,
        "bicycle": 2,
        "car": 3,
        "van": 4,
        "truck": 5,
        "tricycle": 6,
        "awning-tricycle": 7,
        "bus": 8,
        "motor": 9,
    }

    def convert_label(image_name, image_width, image_height, orig_label_dir, save_dir):
        """Converts a single image's DOTA annotation to YOLO OBB format and saves it to a specified directory."""
        orig_label_path = orig_label_dir / f"{image_name}.txt"
        save_path = save_dir / f"{image_name}.txt"

        with orig_label_path.open("r") as f, save_path.open("w") as g:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) < 9:
                    continue
                class_name = parts[8]
                class_idx = class_mapping[class_name]
                coords = [float(p) for p in parts[:8]]
                normalized_coords = [
                    coords[i] / image_width if i % 2 == 0 else coords[i] / image_height for i in range(8)
                ]
                formatted_coords = ["{:.6g}".format(coord) for coord in normalized_coords]
                g.write(f"{class_idx} {' '.join(formatted_coords)}\n")

    for phase in ["train", "val"]:
        image_dir = dota_root_path / "images" / phase
        orig_label_dir = dota_root_path / "labels" / f"{phase}_original"
        save_dir = dota_root_path / "labels" / phase

        save_dir.mkdir(parents=True, exist_ok=True)

        image_paths = list(image_dir.iterdir())
        for image_path in tqdm(image_paths, desc=f"Processing {phase} images"):
            if image_path.suffix != ".jpg":
                continue
            image_name_without_ext = image_path.stem
            img = cv2.imread(str(image_path))
            h, w = img.shape[:2]
            convert_label(image_name_without_ext, w, h, orig_label_dir, save_dir)

In [ ]:
#from ultralytics.data.converter import convert_dota_to_yolo_obb

my_convert_dota_to_yolo_obb('/content/drive/MyDrive/yolodump/DOTA')

Processing val images: 100%|██████████| 500/500 [03:57<00:00,  2.10it/s]


# Install Yolo v8

In [ ]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()
!yolo mode=checks

ModuleNotFoundError: No module named 'ultralytics'

# Training the Model

In [ ]:
from ultralytics import YOLO

# Load a DOTA-pretrained YOLOv8n model
model = YOLO('yolov8x-obb.pt')

# Display model information (optional)
model.info()

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data='/content/drive/MyDrive/yolodump/yolo_data.yaml', epochs=1, imgsz=1024, batch=2, device=0)

# Run inference with the YOLOv8n model on the 'bus.jpg' image
# results = model('path/to/bus.jpg')

YOLOv8x-obb summary: 390 layers, 69493056 parameters, 0 gradients, 264.0 GFLOPs
Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolov8x-obb.pt, data=/content/drive/MyDrive/yolodump/yolo_data.yaml, epochs=1, time=None, patience=100, batch=2, imgsz=1024, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frame

100%|██████████| 755k/755k [00:00<00:00, 15.9MB/s]


Overriding model.yaml nc=15 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytic

100%|██████████| 6.23M/6.23M [00:00<00:00, 72.3MB/s]


AMP: checks passed ✅


...

val: New cache created: /content/drive/MyDrive/yolodump/DOTA/labels/val.cache
Plotting labels to runs/obb/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 103 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1024 train, 1024 val
Using 2 dataloader workers
Logging results to runs/obb/train3
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      6.64G     0.6804      1.047      1.308         46       1024: 100%|██████████| 1176/1176 [09:54<00:00,  1.98it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 250/250 [01:22<00:00,  3.02it/s]


                   all        500       9617       0.66      0.377      0.392      0.189

1 epochs completed in 0.192 hours.
Optimizer stripped from runs/obb/train3/weights/last.pt, 139.5MB
Optimizer stripped from runs/obb/train3/weights/best.pt, 139.5MB

Validating runs/obb/train3/weights/best.pt...
Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-obb summary (fused): 287 layers, 69458721 parameters, 0 gradients, 263.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 250/250 [01:16<00:00,  3.27it/s]


                   all        500       9617       0.66      0.377      0.392      0.189
            pedestrian        500        369          1          0      0.046     0.0159
                people        500         65          0          0          0          0
               bicycle        500        173          0          0          0          0
                   car        500        583      0.891      0.952      0.942      0.357
                   van        500         66          1          0     0.0359     0.0106
                 truck        500        216      0.811      0.972      0.974      0.695
              tricycle        500          7          1          0          0          0
       awning-tricycle        500       1375      0.997       0.94      0.993      0.422
                   bus        500          1          0          0          0          0
                 motor        500       6762      0.897      0.901      0.927      0.391
Speed: 0.6ms preproce

Test model on image

In [ ]:
from ultralytics import YOLO

# Load the trained YOLOv8 model
model = YOLO('path/to/your/trained/model.pt')

# Specify the path to your test image
image_path = 'path/to/your/test/image.jpg'

# Perform prediction on the test image
results = model.predict(source=image_path, save=True, save_txt=True, conf=0.25)

# Print the predicted bounding boxes and class labels
for result in results:
    boxes = result.boxes
    for box in boxes:
        cls_name = result.names[box.cls.item()]
        conf = box.conf.item()
        print(f"Class: {cls_name}, Confidence: {conf:.2f}, Bounding Box: {box.xyxy}")

In [ ]:
# model = YOLO('/path/to/your/local/folder/run_name/weights/best.pt')

Test model on video

In [ ]:
from ultralytics import YOLO

# Load the trained YOLOv8 model
model = YOLO('path/to/your/trained/model.pt')

# Specify the path to your input video
video_path = 'path/to/your/input/video.mp4'

# Perform prediction on the video
results = model.predict(source=video_path, save=True, conf=0.25)

# Print the predicted bounding boxes and class labels for each frame
for result in results:
    boxes = result.boxes
    for box in boxes:
        cls_name = result.names[box.cls.item()]
        conf = box.conf.item()
        print(f"Class: {cls_name}, Confidence: {conf:.2f}, Bounding Box: {box.xyxy}")

In [ ]:
from ultralytics import YOLO

# Load the trained YOLOv8 model
model = YOLO('best.pt')

# Specify the path to your input video
video_path = '9.mp4'

# Perform prediction on the video
results = model.predict(source=video_path, save=True, conf=0.25)

# Iterate over the predicted results for each frame
for result in results:
    # Get the boxes, class names, and confidence scores
    boxes = result.boxes
    class_names = result.names
    confidences = boxes.conf

    # Draw blue bounding boxes on the frame
    result_frame = result.plot(boxes=boxes, labels=class_names, conf=confidences, line_width=2, font_size=12, box_color=(255, 0, 0))

# Save the processed video with blue bounding boxes
processed_video_path = 'predited9.mp4'
results.save(processed_video_path)